Evaluar las derivas del portico debido a las cargas sismicas segun la NSR-10, por medio de un análisis modal espectral y un análisis por fuerza horizontal equivalente 
$$ $$

Realice el análisis:

1) usando un programa o manual

2) Usando un programa software comercial (SAP,  ETABS,  MIDAS,  ROBOT)

Solucion:
$$ $$
Se usa el software Python para realizar el analisis modal y FHE, a continuacion se procede a mostrar y explicar el codigo usado
$$ $$
Se cargan las librerias necesarias para el analisis

In [1]:
import openpyxl as op
import numpy as np
import math as m
import matplotlib.pyplot as plt
import xlsxwriter
import sympy
from sympy.solvers import solve
from sympy import Matrix
from sympy import Symbol

Se crean las funciones necesarias 

In [10]:
#funcion para crear la matriz de rigidez total

def KGlobal(GL, Nelementos):
    
    #se muestran las hojas de excel que hay en el archivo
    print(excel_document.get_sheet_names())
    
    #se pide el nombre de la hoja con los datos
    CMR = excel_document.get_sheet_by_name(input('nombre de la hoja '))
    
    #arreglo que almacenara los datos de excel
    elementos = [[0]*12]*Nelementos  
    #matriz que almacenara la matriz total
    KT = np.matrix(np.zeros((GL,GL)))         
    
    #ciclo que armara las matrices
    for i in range(Nelementos):
        
        for x in range(12):
            
            #Se leen los datos de excel
            elementos[i][x] = CMR.cell(row=2+i, column=1+x).value   
            
#Variables que almacenan los datos de cada elemento de la estructura
        A = elementos[i][1]
        E = elementos[i][2]
        I = elementos[i][3]
        L = elementos[i][4]
        theta = elementos[i][5]*m.pi/180
        Nx, Ny, Nz = elementos[i][6:9]
        Fx, Fy, Fz = elementos[i][9:12]
        y = m.sin(theta)
        x = m.cos(theta)
        
    #se crean una matriz local en terminos globales para cada elemento
        k = np.zeros((GL,GL))
        k[Nx,Nx] = (A * E * x**2 /  L) + (12 * E * I * y**2 / L**3)
        k[Ny,Nx] = k[Nx,Ny] = (A * E / L - 12 * E * I / L**3) * y * x
        k[Nz,Nx] = k[Nx,Nz] = -6 * E * I * y / L**2
        k[Fx,Nx] = k[Nx,Fx]=-((A * E * x**2 / L)+(12 * E * I *y**2/L**3))
        k[Fy,Nx] = k[Nx,Fy] = -((A * E / L - 12 * E * I / L**3) * y * x)
        k[Fz,Nx] = k[Nx,Fz] = -6 * E * I * y / L**2
        k[Ny,Ny] = ( A * E * y**2 / L) + (12 * E * I * x**2 / L**3)
        k[Nz,Ny] = k[Ny,Nz] = 6 * E * I * x / L**2
        k[Fx,Ny] = k[Ny,Fx] = -((A * E / L - 12 * E * I / L**3) * y * x)
        k[Fy,Ny] = k[Ny,Fy]=-((A * E * y**2 / L)+(12 * E * I *x**2/L**3))
        k[Fz,Ny] = k[Ny,Fz] = 6 * E * I * x / L**2
        k[Nz,Nz] = 4 * E * I / L
        k[Fx,Nz] = k[Nz,Fx] = 6 * E * I * y / L**2
        k[Fy,Nz] = k[Nz,Fy] = -(6 * E * I * x / L**2)
        k[Fz,Nz] = k[Nz,Fz] = 2 * E * I / L
        k[Fx,Fx] = (A * E * x**2 / L) + (12 * E * I * y**2 / L**3)
        k[Fy,Fx] = k[Fx,Fy] = (A * E / L - 12 * E * I / L**3) * y * x
        k[Fz,Fx] = k[Fx,Fz] = 6 * E * I * y / L**2
        k[Fy,Fy] = (A * E * y**2 / L) + (12 * E * I * x**2 / L**3)
        k[Fz,Fy] = k[Fy,Fz] = -(6 * E * I * x / L**2)
        k[Fz,Fz] = 4 * E * I / L
        
        #adhiere cada matriz local en terminos globales a la matriz total
        KT = KT + k            
    return KT

#-------------------------------------------------------------------------

#Funcion para condensar la matriz de rigidez

def Kequivalente(GL, Nelementos, d0, Fp):
    
    #arreglo que almacenara los desplazamiento 0
    dN0 = list(range(GL))            
    
    #se usa la funcion para crear la matriz total
    KT = KGlobal(GL, Nelementos)
    
    #se selecciona que filas y columnas eliminar
    dN0 = list(filter(lambda x: x not in d0, dN0))
    
    #se reduce la matriz
    KT = KT[dN0,:]
    KT = KT[:,dN0]
    
    #se transforma ese GDL de terminos globales a locales
    F =[]
    for i in Fp:
        l = 0
        for j in dN0:
            if i == j:
                F = F + [l]
            l = l+ 1 
    F = sorted(F)
    
    #se crea un arreglo con las filas y columnas ordenas 
    c = list(range(len(dN0)))
    [c.remove(j) for j in F]
    c = F + c
    
    #se reacomoda la matriz global reducida
    KT = KT[c,:]
    KT = KT[:,c]
    
    #se seleccionas lo principal y secundario
    KPP = KT[ 0 : len(Fp) , 0 : len(Fp) ]
    KPS = KT[ 0 : len(Fp) , len(Fp): ]
    KSP = KT[ len(Fp): , 0 : len(Fp) ]
    KSS = KT[ len(Fp): , len(Fp): ]
    
    #se opera para hallar la rigidez equivalente
    KEQ = KPP - (KPS * (KSS.I*KSP))  
    
    return KEQ, KSS, KSP, KPP

#------------------------------------------------------------

#Funcion que hallara realizara el analisis modal espectral

def modal(GL, Ne, d0, Fp, masa, Np, modos, Av, Aa, I, Fv, Fa):
    
    #se usa la funcion para crear la matriz de rigidez
    K, KSS, KSP, KPP = Kequivalente(GL, Ne, d0, Fp)
    
    #se crea la matriz de masas
    m1 = np.eye(Np)*masa
    
    #se crea la variable omega para las frecuencias angulares
    w = Symbol('w')
    
    #se plantea y soluciones el determinate para hallar omega
    c = Matrix(K) - w*Matrix(m1)
    d = c.det()
    d = solve(d,w)
    d = np.array([[sympy.re(i)] for i in d])

    #arreglo que guarda los modos de vibracion
    h = []
    #proceso para crear los modos de vibracios
    for i in range(Np):
        e = (K - d[i]*m1) * Matrix([1] + modos)
        g = solve(e[0:len(modos)], modos)
        h = h + [[1] + [float(g[j]) for j in modos]]
        
    h = np.array(h)
    
    #se normaliza los modos para creas la matriz de phi
    phi = np.array([h[i] / m.sqrt(sum(h[i]**2 * masa)) for i in range(Np)])
    
    #matriz de frecuancias w^2
    w = np.eye(Np)*d
    
    #se crean las ecuaciones 
    ecua = Matrix(inc[0] + inc[1]*d.T)

    #se crea el vector de contribucion de masas
    eta = - (np.matrix(phi) * np.matrix(masa).T)
    eta = np.abs(eta)
    
    #se calcula el vector de periodos
    T = [2*m.pi/m.sqrt(d[i]) for i in range(Np)]
    
    #se calculas los periodos del espectro
    T0 = 0.1 * (Av*Fv)/(Aa*Fa)
    Tc = 0.48 * (Av * Fv / Aa* Fa)
    Tl = 2.4 * Fv
    
    Sa = []
    
    #se calcula el vector de aceleraciones espectrales
    for i in T:
        if i < T0:
            Sa = Sa + [2.5*Aa*Fa*I*(0.4+ (0.6*i/T0))]
        if i < Tc and i > T0:
            Sa = Sa + [2.5*Aa*Fa*I]
        if i > Tc and i<Tl:
            Sa = Sa + [1.2*Av*Fv*I/i]
        if i > Tl:
            Sa = Sa + [1.2*Av*Fv*I*Tl / i**2]
    
    Sa = np.array(Sa).T 
    
    Sd = np.array([Sa[i] * 9.804 / np.array(d[i]) for i in range(Np)])
    
    #se calcula los Z maximos
    Z = Sd.T * np.array(eta).T
    Z = np.array(Z).T
    
#se calcula los desplazamientos principales maximos para cada modo

    dp1 = np.array([phi.T[:,i] * Z[i,0] for i in range(Np)]).T
    
    #se halla el desplazamiento absoluto (CQC)
    dp = np.matrix([m.sqrt(sum(dp1[i]**2)) for i in range(Np)]).T
    
    #se calcula los desplazamientos secundarios maximos para modo
    ds = -(KSS.I*KSP)*np.matrix(dp)
    
    #se presentan los datos relevantes
    print(' ')
    print('MATRIZ DE MASAS')
    print(m1)
    print(' ')
    print('MATRIZ DE RIGIDEZ EQUIVALENTE')
    print(K)
    print(' ')
    print('MATRIZ DE MODOS DE FRECUENCIAS ANGULARES')
    print(w)
    print(' ')
    print('VECTOR DE PERIODOS')
    print(np.matrix(T).T)
    print(' ')
    print('VECTOR DE ACELERACIONES ESPECTRALES')
    print(np.matrix(Sa).T)
    print(' ')
    print('MATRIZ DE MODOS')
    print(h)
    print(' ')
    print('MATRIZ DE MODOS NORMALIZADOS')
    print(phi.T)
    print(' ')
    print('ECUACIONES DESACOPLADAS')
    [print(ecua[i], ' = ', eta[i]) for i in range(len(masa))]
    print(' ')
    print('COEFICIENTES DE PARTICIPACION MODAL')
    print(eta)
    print(' ')
    print('VALORES MAXIMOS DE Z ')
    print(Z)
    print(' ')
    print('DESPLAZAMIENTO PRINCIPALES')
    print(dp)
    print(' ')
    print('DESPLAZAMIENTO SECUNDARIO')
    print(ds)
    return

#----------------------------------------------------------------

#funcion para hacer el analisis de FHE

def FHE(masa, niveles, Sa, k, GL, Ne, d0, Fp):
    
    #se usa la funcion para crear la matriz de rigidez
    K, KSS, KSP, KPP = Kequivalente(GL, Ne, d0, Fp)
    
    #se halla el cortante basal
    Vs = sum(masa) * 9.81 * Sa
    
    #se crea un arreglo con las masas por la altura del nivel
    mhk = [masa[i]*niveles[i]**k for i in range(len(masa))]
    
    #se crea un arreglo con los coeficifientes Cvx
    Cvx = [mhk[i] / sum(mhk) for i in range(len(masa)) ]
    
    #se crea un arreglo con las fuerza de cada nivel
    Fx = [[Cvx[i] * Vs] for i in range(len(masa))]
    
    #se hallan los desplzamientos principlaes
    dp = K.I*Fx
    #se hallan los desplazamientos secundarios
    ds = (-(KSS.I*KSP))*dp
    
    #se presentan los resultados relevenate
    print(' ')
    print('FUERZAS EN LOS ENTREPISOS')
    print(np.matrix(Fx).T)
    print(' ')
    print('DESPLAZAMIENTOS PRINCIPALES')
    print(dp)
    print(' ')
    print('DESPLAZAMIENTOS SECUNDARIOS')
    print(ds)
    
    return 

#----------------------------------------------------------------------

#Funcion para guardar matrices en excel

def Guardar(KT, name):
    workbook = xlsxwriter.Workbook(name) 
    worksheet = workbook.add_worksheet()
    row = 0
    for col, data in enumerate(np.array(KT)):
        worksheet.write_column(row, col, data)
    workbook.close()


A continuacion se emplearan las funciones para resolver los analisis de la estrutura

In [11]:
#Se carga el archivo de excel con los datos de los elementos
excel_document = op.load_workbook('Final_dinamica.xlsx')    

se ingresan los datos necesarios para armar las matriz de rigidez y condesarla

In [12]:
#Numero total de elementos en la estructura
Ne = 11 
#Numero total de grados de libertad
GL = 30 
#lista de los desplazamientos que son cero o irrelevantes
d0 = [0,1,2,6,7,8,12,13,14,4,10,16,19,22,25,28]  
#grado de libertad de las fuerza principal
Fp = [9,21,27]

se ingresa los datos necesarios para el analisis modal, en necesario ingresar los datos en orden ascendente primero el piso 1,2,3...

In [13]:
#se crean las incongnitas de el analisis modal
a2, a3 = sympy.symbols('a2, a3')
ä1, a1 =  sympy.symbols('ä1, a1') 
ä2, a2 =  sympy.symbols('ä2, a2') 
ä3, a3 =  sympy.symbols('ä3, a3')
t = sympy.symbols('t')

#numero de niveles
Np = 3
#lista de masa de los niveles en toneladas y ascendiente
masa = [24.4648318043, 24.4648318043, 1.01936799185]
#lista con el nombre las incognitas de modos de vibracion
#no tener encuenta el modo a1 puesto que este se supone 1
modos = [a2, a3]
#matriz con los nombre de las incognitas de las ecuaciones diferenciales 
inc = [[ä1, ä2, ä3], [a1, a2, a3]]

#datos de para el espectro de diseño de la NSR-10
Av= 0.25
Fa = 1.15
Fv = 1.55
I = 1
Aa = 0.25

Se usa la funcion "modal" para solucionar el sistema y se muestra los resultados relevantes, la comparacion con el software se hara mas adelante

In [14]:
modal(GL, Ne, d0, Fp, masa, Np, modos, Av, Aa, I, Fv, Fa)

['Ejemplo1', 'final_dinamica']
nombre de la hoja final_dinamica
 
MATRIZ DE MASAS
[[ 24.4648318    0.           0.        ]
 [  0.          24.4648318    0.        ]
 [  0.           0.           1.01936799]]
 
MATRIZ DE RIGIDEZ EQUIVALENTE
[[ 30324.23685301 -14904.78053074   1522.09821652]
 [-14904.78053074  15653.69135093  -4187.49968186]
 [  1522.09821652  -4187.49968186   2917.26169792]]
 
MATRIZ DE MODOS DE FRECUENCIAS ANGULARES
[[134.018824005052 0 0]
 [0 1344.59318680135 0]
 [0 0 3262.56953017538]]
 
VECTOR DE PERIODOS
[[ 0.54274642]
 [ 0.17135012]
 [ 0.11000185]]
 
VECTOR DE ACELERACIONES ESPECTRALES
[[ 0.71875   ]
 [ 0.71875   ]
 [ 0.63946156]]
 
MATRIZ DE MODOS
[[  1.           2.07826601   2.58236286]
 [  1.          -0.37732612  -2.00575338]
 [  1.          -1.43643649  18.45098231]]
 
MATRIZ DE MODOS NORMALIZADOS
[[ 0.08545734  0.17664157  0.04868058]
 [ 0.17760309 -0.06665148 -0.06992656]
 [ 0.22068186 -0.35429942  0.89820456]]
 
ECUACIONES DESACOPLADAS
134.018824005052*a

se ingresan los datos para realizar el analisis de FHE

In [15]:
#lista con las alturas de cada nivel
niveles = [3,6,8]
#aceleracion de el terreno
Sa = 0.71875
#cosntante k
k = 1

se usa la funcion FHE para realizar el analis de FHE

In [16]:
FHE(masa, niveles, Sa, k, GL, Ne, d0, Fp)

['Ejemplo1', 'final_dinamica']
nombre de la hoja final_dinamica
 
FUERZAS EN LOS ENTREPISOS
[[ 113.203125  226.40625    12.578125]]
 
DESPLAZAMIENTOS PRINCIPALES
[[ 0.03365255]
 [ 0.06974267]
 [ 0.08686332]]
 
DESPLAZAMIENTOS SECUNDARIOS
[[ 0.03353939]
 [-0.01034055]
 [-0.00720356]
 [ 0.03355555]
 [-0.01048893]
 [ 0.0696434 ]
 [-0.00736768]
 [-0.00369562]
 [ 0.06960203]
 [-0.00651024]
 [-0.0092311 ]]
